In [1]:
import os
import json
import requests
import base64
from datetime import date
import re
import time
from pprint import pprint
from openpyxl import load_workbook
import xlsxwriter
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pyotp import TOTP
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from seleniumbase import Driver

In [2]:
brand = "EBC"
print(f"Automated fulfillment started {brand}")

Automated fulfillment started EBC


In [3]:
# Gorgias Login Details

GORGIAS_URL = "https://mlperformance-1.gorgias.com"
GORGIAS_OD_USERNAME = 'amanina@mlperformance.co.uk'
GORGIAS_OD_API_KEY = 'a40765a261a5096b6c4f04d16b25bf655cb77edb6c397ee88b8267480caa54b8'

AUTHORIZATION = base64.b64encode(
    f"{GORGIAS_OD_USERNAME}:{GORGIAS_OD_API_KEY}".encode("utf-8")
).decode("utf-8")

HEADERS = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Basic {AUTHORIZATION}",
}
# Shopify credentials
CREDS_FILE = r"C:\Users\User\Desktop\Automated Fulfillment\constant\login_detail.json"
SHOPIFY_ADMIN_URL = "https://admin.shopify.com/store/mlperformance/purchase_orders"
try:
    with open(CREDS_FILE, "r") as f:
        CREDS = json.load(f)
    SHOPIFY_EMAIL = CREDS["login_Jarvis"]["Email"]
    SHOPIFY_PASSWORD = CREDS["login_Jarvis"]["Password"]
    SHOPIFY_SECRET_KEY = CREDS["login_Jarvis"]["Secret Key"]
except FileNotFoundError:
    print("Credentials file not found. Using placeholder credentials.")
    SHOPIFY_EMAIL = "jarvis@mlperformance.co.uk"
    SHOPIFY_PASSWORD = "Te@mMLP2024!"
    SHOPIFY_SECRET_KEY = "Y55XYVN4MULN372ISPYVK4B7WRG22R6N"

# print(HEADERS)

In [4]:
# Supplier details
supplier_list = {
    "EBC": {
        "supplier_id": 153616543,  # supplier id in po shopify
        "supplier_email": ["pkginfo@ups.com"],
        "customer_ids": [175689442],  # customer id in gorgias
        "subject_pattern": "UPS Shipping Notification"
    }
}
print("Supplier list defined:", supplier_list)

Supplier list defined: {'EBC': {'supplier_id': 153616543, 'supplier_email': ['pkginfo@ups.com'], 'customer_ids': [175689442], 'subject_pattern': 'UPS Shipping Notification'}}


In [5]:
def generate_ticket_list(tag_id):
    subject_pattern = supplier_list["EBC"]["subject_pattern"]
    customer_ids = supplier_list["EBC"]["customer_ids"]
    ticket_id_list = []

    for customer_id in customer_ids:
        supplier_url = f"{GORGIAS_URL}/api/tickets?customer_id={customer_id}&order_by=created_datetime%3Adesc&limit=100"
        #supplier_url = f"{GORGIAS_URL}/api/tickets?customer_id={customer_id}"
        print(supplier_url)
        response = requests.get(supplier_url, headers=HEADERS).json()
        # pprint(response)
        # cursor = response["meta"]["next_cursor"]

        # supplier_url_next = f"{supplier_url}&cursor={cursor}"

        # for data in response["data"]:
        #     print(data["id"])
        #     print(data["subject"])

        
        #     # print(data["tags"])

        #     if subject_pattern in data["subject"]:
        #         print("yes")


        
        ticket_id_list.extend(
            [
                data["id"]
                for data in response["data"]
                if (
                    re.search(subject_pattern, data["subject"])
                    # and any(tag_id in all_tag.values() for all_tag in data["tags"])
                    # and data["status"] == "open"
                )
            ]
        )
    return ticket_id_list

ticket_id_list = generate_ticket_list('711016')
ticket_id_list

https://mlperformance-1.gorgias.com/api/tickets?customer_id=175689442&order_by=created_datetime%3Adesc&limit=100


[166230623,
 164974177,
 164974152,
 164974211,
 164974150,
 164787579,
 164259905,
 164259750,
 163896555,
 163884501,
 163884496,
 163760165,
 163760161,
 163760157,
 163760144,
 163760137,
 163734766,
 161975170,
 160701061,
 160701047,
 160528866,
 160528863,
 160528862,
 160528855,
 160514246,
 160476143,
 158897912,
 158897903,
 158897898,
 158262440,
 158107439]

In [6]:
# Fetch a single ticket from Gorgias
def get_single_ticket(ticket_id):
    url = f"{GORGIAS_URL}/api/tickets/{ticket_id}"
    response = requests.get(url, headers=HEADERS)
    
    if response.status_code != 200:
        print(f"Failed to fetch ticket {ticket_id}: {response.status_code}")
        print(f"Error message: {response.text}")
        return None
    
    try:
        response_json = response.json()
        print(f"Raw API response for ticket {ticket_id}:")
        #pprint(response_json)  # Print
        
        # Try different response structures
        ticket = response_json.get("data") or response_json
        if not ticket:
            print(f"Ticket {ticket_id}: No ticket data found in response.")
            return None
        
        print(f"Successfully fetched ticket {ticket_id}.")
        return ticket
    except ValueError as e:
        print(f"Ticket {ticket_id}: Failed to parse JSON response: {str(e)}")
        print(f"Raw response: {response.text}")
        return None

ticket_data = {}

for single_ticket_id in ticket_id_list:
    # Fetch the ticket
    ticket = get_single_ticket(single_ticket_id)
    ticket_data[single_ticket_id] = ticket
    if ticket:
        print(f"Processed ticket data for {single_ticket_id}:")
        # pprint(ticket)
    else:
        print("No ticket data to process.")

Raw API response for ticket 166230623:
Successfully fetched ticket 166230623.
Processed ticket data for 166230623:
Raw API response for ticket 164974177:
Successfully fetched ticket 164974177.
Processed ticket data for 164974177:
Raw API response for ticket 164974152:
Successfully fetched ticket 164974152.
Processed ticket data for 164974152:
Raw API response for ticket 164974211:
Successfully fetched ticket 164974211.
Processed ticket data for 164974211:
Raw API response for ticket 164974150:
Successfully fetched ticket 164974150.
Processed ticket data for 164974150:
Raw API response for ticket 164787579:
Successfully fetched ticket 164787579.
Processed ticket data for 164787579:
Raw API response for ticket 164259905:
Successfully fetched ticket 164259905.
Processed ticket data for 164259905:
Raw API response for ticket 164259750:
Successfully fetched ticket 164259750.
Processed ticket data for 164259750:
Raw API response for ticket 163896555:
Successfully fetched ticket 163896555.
Pr

In [7]:
import re

def extract_shipment_details(ticket):
    """
    Extract shipment details from the ticket's message body
    Returns a dictionary with relevant details or None if extraction fails.
    """
    try:
        # Get the first message's body_text and sender email
        message_body = ticket.get("messages", [{}])[0].get("body_text", "")
        sender_email = ticket.get("messages", [{}])[0].get("from", {}).get("address", "")
        if not message_body:
            print(f"Ticket {ticket['id']}: No message body found.")
            return None

        # Define regex patterns for extraction
        patterns = {
            "reference_num": r"Reference Number 1:\s*(\d+)",
            "tracking_number": r"Tracking Number:\s*([A-Za-z0-9]+)",
            "ship_to": r"Ship To:\s*([\w\s]+)\n([\w\s\d,]+)\n([A-Z]{2})",
            "scheduled_delivery": r"Scheduled Delivery:\s*([\d/]+)",
            "ups_service": r"UPS Service:\s*([\w\s]+)",
            "parcel_weight": r"Parcel Weight:\s*([\d.]+)\s*KGS",
            "number_of_parcels": r"Number of Parcels:\s*(\d+)",
            "from_field": r"From:\s*(FREEMAN AUTOMOTIVE UK LTD|[\w\s]+)"  # Specific for EBC supplier
        }

        shipment_details = {
            "ticket_id": ticket["id"],
            "ticket_link": f"{GORGIAS_URL}/app/ticket/{ticket['id']}",
            "sender_email": sender_email
        }

        # Extract details using regex
        for key, pattern in patterns.items():
            match = re.search(pattern, message_body, re.MULTILINE)
            shipment_details[key] = match.group(1).strip() if match else "Not found"

        print(f"Extracted details for ticket {ticket['id']}:\n{shipment_details}")
        return shipment_details
    except Exception as e:
        print(f"Ticket {ticket['id']}: Failed to extract shipment details: {str(e)}")
        return None

ticket_info = []
for ticket_id, ticket in ticket_data.items():
    if ticket:
        details = extract_shipment_details(ticket)
        if details:
            ticket_info.append(details)
    else:
        print(f"Ticket {ticket_id}: No valid ticket data to process.")

print("\nCollected Ticket Info:")
for info in ticket_info:
    print(info)

Extracted details for ticket 166230623:
{'ticket_id': 166230623, 'ticket_link': 'https://mlperformance-1.gorgias.com/app/ticket/166230623', 'sender_email': '', 'reference_num': 'Not found', 'tracking_number': '1Z57944V6851066954', 'ship_to': 'Not found', 'scheduled_delivery': 'Not found', 'ups_service': 'UPS STANDARD  \n Number of Parcels', 'parcel_weight': 'Not found', 'number_of_parcels': '1', 'from_field': 'BREMSEN TECHNIK'}
Extracted details for ticket 164974177:
{'ticket_id': 164974177, 'ticket_link': 'https://mlperformance-1.gorgias.com/app/ticket/164974177', 'sender_email': '', 'reference_num': '896199', 'tracking_number': '1Z6A33636858741760', 'ship_to': 'nayeem nayeem miah \n20 more close', 'scheduled_delivery': 'Not found', 'ups_service': 'UPS STANDARD  \n Number of Parcels', 'parcel_weight': '21.0', 'number_of_parcels': '1', 'from_field': 'FREEMAN AUTOMOTIVE UK LTD'}
Extracted details for ticket 164974152:
{'ticket_id': 164974152, 'ticket_link': 'https://mlperformance-1.gorg

In [8]:
def filter_tickets(ticket_info, supplier_name="EBC UPS"):
    """
    Filter tickets based where from_field is 'FREEMAN AUTOMOTIVE UK LTD'.
    """
    filtered_tickets = []
    for ticket in ticket_info:
        ticket_id = ticket["ticket_id"]
        # Check if from_field matches 'FREEMAN AUTOMOTIVE UK LTD'
        if ticket["from_field"] == "FREEMAN AUTOMOTIVE UK LTD":
            filtered_tickets.append(ticket)
            print(f"Ticket {ticket_id}: Matches EBC criteria (From: FREEMAN AUTOMOTIVE UK LTD), added to filtered list.")
        else:
            print(f"Ticket {ticket_id}: Does not match(from_field: {ticket['from_field']}), skip.")
    
    print(f"\nFound {len(filtered_tickets)} tickets matching 'From: FREEMAN AUTOMOTIVE UK LTD' for '{supplier_name}'.")
    for ticket in filtered_tickets:
        print(f"Ticket ID: {ticket['ticket_id']}")
    
    return filtered_tickets

filtered_tickets = filter_tickets(ticket_info, supplier_name="EBC UPS")

Ticket 166230623: Does not match(from_field: BREMSEN TECHNIK), skip.
Ticket 164974177: Matches EBC criteria (From: FREEMAN AUTOMOTIVE UK LTD), added to filtered list.
Ticket 164974152: Matches EBC criteria (From: FREEMAN AUTOMOTIVE UK LTD), added to filtered list.
Ticket 164974211: Matches EBC criteria (From: FREEMAN AUTOMOTIVE UK LTD), added to filtered list.
Ticket 164974150: Matches EBC criteria (From: FREEMAN AUTOMOTIVE UK LTD), added to filtered list.
Ticket 164787579: Does not match(from_field: CO ORD SPORT LIMITED  
 Tracking Number), skip.
Ticket 164259905: Does not match(from_field: BREMSEN TECHNIK), skip.
Ticket 164259750: Does not match(from_field: BREMSEN TECHNIK), skip.
Ticket 163896555: Does not match(from_field: PURETURBOS EU BVBA  
 Tracking Number), skip.
Ticket 163884501: Does not match(from_field: BREMSEN TECHNIK), skip.
Ticket 163884496: Does not match(from_field: BREMSEN TECHNIK), skip.
Ticket 163760165: Matches EBC criteria (From: FREEMAN AUTOMOTIVE UK LTD), added

In [9]:
def print_freeman_tickets(ticket_info):
    """
    Print reference_num, tracking_number, and ticket_link for tickets where from_field is 'FREEMAN AUTOMOTIVE UK LTD'.
    """
    print("\nTickets for EBC UPS")
    print("-" * 50)
    freeman_tickets = [ticket for ticket in ticket_info if ticket["from_field"] == "FREEMAN AUTOMOTIVE UK LTD"]
    
    if not freeman_tickets:
        print("No tickets found from FREEMAN AUTOMOTIVE UK LTD.")
        return
    
    for ticket in freeman_tickets:
        print(f"Ticket ID: {ticket['ticket_id']}")
        print(f"Reference Number: {ticket['reference_num']}")
        print(f"Tracking Number: {ticket['tracking_number']}")
        print(f"Ticket Link: {ticket['ticket_link']}")
        print("-" * 50)
    
    print(f"Total Tickets for EBC UPS: {len(freeman_tickets)}")

# Call the function with ticket_info
print_freeman_tickets(ticket_info)


Tickets for EBC UPS
--------------------------------------------------
Ticket ID: 164974177
Reference Number: 896199
Tracking Number: 1Z6A33636858741760
Ticket Link: https://mlperformance-1.gorgias.com/app/ticket/164974177
--------------------------------------------------
Ticket ID: 164974152
Reference Number: 896194
Tracking Number: 1Z6A33636860579905
Ticket Link: https://mlperformance-1.gorgias.com/app/ticket/164974152
--------------------------------------------------
Ticket ID: 164974211
Reference Number: 896193
Tracking Number: 1Z6A33636859525733
Ticket Link: https://mlperformance-1.gorgias.com/app/ticket/164974211
--------------------------------------------------
Ticket ID: 164974150
Reference Number: 896195
Tracking Number: 1Z6A33636860077724
Ticket Link: https://mlperformance-1.gorgias.com/app/ticket/164974150
--------------------------------------------------
Ticket ID: 163760165
Reference Number: 894824
Tracking Number: 1Z6A33636859112045
Ticket Link: https://mlperformance

In [10]:
# Function to initialize Selenium browser for Shopify login
def initiate_browser(email, password, secret_key=None):
    browser = Driver(incognito=True)
    browser.get("https://admin.shopify.com/store/mlperformance/orders?inContextTimeframe=none")
    wait = WebDriverWait(browser, 30)
    
    # Enter email
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="account_email"]'))).send_keys(email)
    time.sleep(2)
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".captcha__submit"))).click()
    
    # Enter password
    time.sleep(2)
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ui-password__input"))).send_keys(password)
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".captcha__submit"))).click()
    
    # Handle 2FA if secret_key is provided
    if secret_key:
        from pyotp import TOTP
        auth_field = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input#account_tfa_code")))
        totp = TOTP(secret_key)
        token = totp.now()
        auth_field.send_keys(token)
        time.sleep(3)
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ui-button--full-width"))).click()
    
    return browser

print("Browser initialization function defined.")

# Initialize browser for Shopify
browser = initiate_browser(SHOPIFY_EMAIL, SHOPIFY_PASSWORD, SHOPIFY_SECRET_KEY)
if browser:
    print("Browser initialized and logged into Shopify.")
else:
    print("Browser initialization failed. Cannot proceed with PO search.")

Browser initialization function defined.


*** chromedriver to download = 137.0.7151.55 (Latest Stable) 

https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.55/win64/chromedriver-win64.zip ...
Download Complete!

Extracting ['chromedriver.exe'] from chromedriver-win64.zip ...
Unzip Complete!

The file [chromedriver.exe] was saved to:
C:\Users\User\AppData\Roaming\Python\Python312\site-packages\seleniumbase\drivers\chromedriver.exe

Making [chromedriver.exe 137.0.7151.55] executable ...
[chromedriver.exe 137.0.7151.55] is now ready for use!

Browser initialized and logged into Shopify.


In [11]:
def get_po_link(reference_num, browser):
    """
    Search for PO in Shopify by reference number and return the PO link.
    """
    if not browser:
        print(f"Cannot search for PO with reference {reference_num}: Browser not initialized.")
        return "Browser not initialized"
    
    if reference_num == "Not found":
        print(f"Cannot search for PO: Reference number is 'Not found'.")
        return "PO not found"
    
    query_ref = f"0000{reference_num}"  # add "0000" for EBC Ref number
    search_url = f"{SHOPIFY_ADMIN_URL}?selectedView=all&query={query_ref}"
    
    try:
        browser.get(search_url)
        wait = WebDriverWait(browser, 10)
        po_link = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[data-primary-link="true"]'))
        ).get_attribute("href")
        print(f"Found PO for reference {query_ref}: {po_link}")
        return po_link
    except Exception as e:
        print(f"Error finding PO for reference {query_ref}")
        return "PO not found"

def get_po_tags(po_url, browser):
    """
    Extract tag (Dropship or Warehouse) and order_id (MLP or SCP) from a Shopify PO page.
    """
    if po_url in ["PO not found", "Browser not initialized"]:
        print(f"Cannot fetch tags for PO URL {po_url}: Invalid URL.")
        return "Tag not found", "Order ID not found"
    
    try:
        browser.get(po_url)
        wait = WebDriverWait(browser, 15)
        tag_list = []
        tags = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.Polaris-Tag__Text'))
        )
        for itag in tags:
            shop_tag = itag.get_attribute('innerText')
            tag_list.append(shop_tag)
        
        if 'Dropship' in tag_list:
            tag0 = 'Dropship'
        else:
            tag0 = 'Warehouse'
        print(f"Found tag: {tag0}")
        
        comment_list = []
        comments = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '._CommentText_1x55c_28'))
        )
        for icomment in comments:
            comment = icomment.get_attribute('innerText')
            if 'MLP' in comment or 'SCP' in comment:
                comment_list.append(comment)
        
        if comment_list:
            target_comment = comment_list[-1]
            order_id = ' '.join(word for word in target_comment.split() if 'MLP' in word or 'SCP' in word)
        else:
            order_id = "Order ID not found"
        print(f"Found order ID: {order_id}")
        
        return tag0, order_id
    
    except Exception as e:
        print(f"Error finding tags for PO URL {po_url}: {str(e)}")
        return "Tag not found", "Order ID not found"

def print_freeman_tickets_with_tags(ticket_info, browser):
    """
    Print tickets from FTickets for EBC UPS with their reference_num, tracking_number, 
    ticket_link, PO link, tag, and order_id.
    """
    print("\nTickets for EBC UPS with PO Tags and Order IDs:")
    print("-" * 60)
    freeman_tickets = [ticket for ticket in ticket_info if ticket["from_field"] == "FREEMAN AUTOMOTIVE UK LTD"]
    
    if not freeman_tickets:
        print("No tickets found from Tickets for EBC UPS")
        return
    
    if not browser:
        print("No browser initialized for PO search or tag extraction.")
        for ticket in freeman_tickets:
            print(f"Ticket ID: {ticket['ticket_id']}")
            print(f"Reference Number: {ticket['reference_num']}")
            print(f"Tracking Number: {ticket['tracking_number']}")
            print(f"Ticket Link: {ticket['ticket_link']}")
            print(f"PO Link: Browser not initialized")
            print(f"Tag: Tag not found")
            print(f"Order ID: Order ID not found")
            print("-" * 60)
        print(f"Total Tickets for EBC UPS: {len(freeman_tickets)}")
        return
    
    for ticket in freeman_tickets:
        po_link = get_po_link(ticket["reference_num"], browser)
        tag, order_id = ("Tag not found", "Order ID not found") if po_link in ["PO not found", "Browser not initialized"] else get_po_tags(po_link, browser)
        
        print(f"Ticket ID: {ticket['ticket_id']}")
        print(f"Reference Number: {ticket['reference_num']}")
        print(f"Tracking Number: {ticket['tracking_number']}")
        print(f"Ticket Link: {ticket['ticket_link']}")
        print(f"PO Link: {po_link}")
        print(f"Tag: {tag}")
        print(f"Order ID: {order_id}")
        print("-" * 60)
    
    print(f"Total Tickets for EBC UPS: {len(freeman_tickets)}")

print_freeman_tickets_with_tags(ticket_info, browser)


Tickets for EBC UPS with PO Tags and Order IDs:
------------------------------------------------------------
Found PO for reference 0000896199: https://admin.shopify.com/store/mlperformance/purchase_orders/10236985727
Found tag: Dropship
Found order ID: MLP147885
Ticket ID: 164974177
Reference Number: 896199
Tracking Number: 1Z6A33636858741760
Ticket Link: https://mlperformance-1.gorgias.com/app/ticket/164974177
PO Link: https://admin.shopify.com/store/mlperformance/purchase_orders/10236985727
Tag: Dropship
Order ID: MLP147885
------------------------------------------------------------
Found PO for reference 0000896194: https://admin.shopify.com/store/mlperformance/purchase_orders/10160243071
Found tag: Dropship
Found order ID: MLP144458
Ticket ID: 164974152
Reference Number: 896194
Tracking Number: 1Z6A33636860579905
Ticket Link: https://mlperformance-1.gorgias.com/app/ticket/164974152
PO Link: https://admin.shopify.com/store/mlperformance/purchase_orders/10160243071
Tag: Dropship
O

In [12]:
def get_order_id_link(order_id, browser):
    """
    Search for the order in Shopify by order_id (MLP or SCP) and return the order URL.
    If order_id contains HTML tags (e.g., <h1>), clean it before searching.
    """
    if not browser:
        print("Cannot search for order: Browser not initialized.")
        return "Browser not initialized"
    
    if order_id == "Order ID not found":
        print(f"Cannot search for order: Order ID is 'Order ID not found'.")
        return "Order URL not found"
    
    # Clean order_id by removing HTML tags if present
    clean_order_id = re.sub(r'<[^>]+>', '', order_id).strip()
    if not clean_order_id.startswith('MLP') and not clean_order_id.startswith('SCP'):
        print(f"Invalid order ID format: {clean_order_id}")
        return "Order URL not found"
    
    search_url = f"https://admin.shopify.com/store/mlperformance/orders?query={clean_order_id}"
    try:
        browser.get(search_url)
        wait = WebDriverWait(browser, 10)
        order_link = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[data-test-id="order-row"] a[href*="/orders/"]'))
        ).get_attribute("href")
        print(f"Found order URL for order ID {clean_order_id}: {order_link}")
        return order_link
    except Exception as e:
        print(f"Error finding order URL for order ID {clean_order_id}")
        return "Order URL not found"
    
def get_order_id_link(order_no, browser):
    browser.get(f'https://admin.shopify.com/store/mlperformance/orders?start=MQ%3D%3D&query={order_no}')
    wait = WebDriverWait(browser, 10)
    try:
        order_link = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.Polaris-Link--removeUnderline'))
        ).get_attribute("href")

        seacrh_od_id = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.Polaris-Link--removeUnderline'))
        ).get_attribute("innerText").strip()

        if order_no in seacrh_od_id:
            return order_link
        else:
            return 'Order link not found'
    except:
        return 'Order link not found'
# Search for POs and generate JSON for EBC UPS tickets

def get_order_id_link(order_no, browser):
    browser.get(f'https://admin.shopify.com/store/mlperformance/orders?start=MQ%3D%3D&query={order_no}')
    wait = WebDriverWait(browser, 10)
    try:
        order_link = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.Polaris-Link--removeUnderline'))
        ).get_attribute("href")

        seacrh_od_id = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.Polaris-Link--removeUnderline'))
        ).get_attribute("innerText").strip()

        if order_no in seacrh_od_id:
            return order_link
        else:
            return 'Order link not found'
    except:
        return 'Order link not found'
    
final_data = {}

if filtered_tickets and browser:
    for ticket in filtered_tickets:
        reference_num = ticket["reference_num"]
        ticket_id = ticket["ticket_id"]
        ticket_link = ticket["ticket_link"]
        tracking_num = ticket["tracking_number"]
        po_link = get_po_link(reference_num, browser)
        tag, order_id = get_po_tags(po_link, browser)
        order_url = get_order_id_link(order_id, browser)

        print(f"Ticket ID: {ticket_id}, Reference Number: {reference_num}, Ticket Link: {ticket_link}, Tracking Number: {tracking_num}, PO Link: {po_link}, Tag: {tag}, Order ID: {order_id}, Order URL: {order_url}")
        print('\n')
            
        final_data[ticket_id] = {
            "reference_number": reference_num,
            "tracking_number": tracking_num,
            "po_url": po_link,
            "tag": tag,
            "order_id": re.sub(r'<[^>]+>', '', order_id).strip() if order_id != "Order ID not found" else order_id,
            "order_url": order_url
        }

    with open("po_data-anis.json", "w") as f:
        json.dump(final_data, f, indent=4)
    print("JSON data saved to 'po_data-ebc-ups.json'.")
else:
    if not filtered_tickets:
        print("No filtered tickets (EBC UPS) to process for PO search.")
    if not browser:
        print("No browser initialized for PO search.")

Found PO for reference 0000896199: https://admin.shopify.com/store/mlperformance/purchase_orders/10236985727
Found tag: Dropship
Found order ID: MLP147885
Ticket ID: 164974177, Reference Number: 896199, Ticket Link: https://mlperformance-1.gorgias.com/app/ticket/164974177, Tracking Number: 1Z6A33636858741760, PO Link: https://admin.shopify.com/store/mlperformance/purchase_orders/10236985727, Tag: Dropship, Order ID: MLP147885, Order URL: https://admin.shopify.com/store/mlperformance/orders/11702779347327


Found PO for reference 0000896194: https://admin.shopify.com/store/mlperformance/purchase_orders/10160243071
Found tag: Dropship
Found order ID: MLP144458
Ticket ID: 164974152, Reference Number: 896194, Ticket Link: https://mlperformance-1.gorgias.com/app/ticket/164974152, Tracking Number: 1Z6A33636860579905, PO Link: https://admin.shopify.com/store/mlperformance/purchase_orders/10160243071, Tag: Dropship, Order ID: MLP144458, Order URL: https://admin.shopify.com/store/mlperformance/